van der Waals's Equation of State
==
This notebook intends to present van der Waals's Equation of State([wikipedia](https://en.wikipedia.org/wiki/Van_der_Waals_equation),[nobel prize lecture](http://www.nobelprize.org/nobel_prizes/physics/laureates/1910/waals-lecture.pdf)), and to derive get density for given values of Pressure and Temperature.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as numpy
from scipy import linalg as linalg
from scipy import optimize as optimize

In [3]:
### input

R = 0.08314

#Selecao de equacqo de estado
ede = 'PR'

#Substancia
subs='hexano'

#Parametros de componente puro (K,bar)
Tc = 507.5
Pc = 30.1
ac=0.299

#Informe a temperatura para o calculo de Psat (K)
T = 220.



In [4]:
### Calculate Psat
if ede=='SRK':
    kappa=0.48508+1.55171*ac-0.15613*ac*ac
    a=0.42748*R*R*Tc*Tc/Pc
    teta=a*(1+kappa-kappa*(T/Tc)**0.5)**2
    b=0.08664*R*Tc/Pc
    sig = 0.
    eps = 1.
    dteta = -a*kappa*(-kappa+(1+kappa)*(T/Tc)**0.5)/Tc
    def psia(V):
        return (1/(b*(eps-sig)))*numpy.log((V+eps*b)/(V+sig*b))
elif ede == 'PR':        
    kappa = 0.37464+1.54226*ac-0.26992*ac*ac   
    a = 0.45724*R*R*Tc*Tc/Pc
    teta = a*(1.+kappa-kappa*(T/Tc)**0.5)**2 
    b = 0.0778*R*Tc/Pc
    sig = 1. + 2**0.5
    eps = 1. - 2**0.5
    dteta = kappa*(-kappa+(1+kappa)*(T/Tc)**0.5)*-a/Tc
    def psia(V):
        return (1/(b*(eps-sig)))*numpy.log((V+eps*b)/(V+sig*b))

rho = R*T
mu = 2*(b*R*T*(eps+sig)-teta)
nu = R*T*(b**2)*(4*sig*eps+eps*eps+sig*sig) - b*(teta*(eps+sig-4))
omega=2*(b**3)*R*T*(sig*eps*eps+eps*sig*sig)- 2*(b**2)*(teta*(1-sig-eps))
zeta = R*T*(b**4)*sig*sig*eps*eps-teta*(b**3)*(eps+sig)

plateaus = numpy.roots([rho,mu,nu,omega,zeta])

print("plateaus:", plateaus)


raizes = plateaus[ numpy.where( plateaus > b ) ]


print("Roots:", raizes)

Vli = min(raizes)
Vvi = max(raizes)

Psupi = R*T/(Vvi-b)-teta/((Vvi+sig*b)*(Vvi+eps*b))-0.000000001
Pinf = R*T/(Vli-b)-teta/((Vli+sig*b)*(Vli+eps*b))

if Pinf > 0:
    Pinfi = Pinf+0.000000001
else:
    Pinfi = 0   +0.000000001

print("Vli:",Vli)
print("Vvi:",Vvi)
print("Psupi:",Psupi)
print("Pinfi:",Pinfi)     

def fps(P):
    alfa = (sig+eps-1)*b-R*T/P
    beta = sig*eps*b**2-(b+R*T/P)*(sig+eps)*b + teta/P
    gamma = -(b+R*T/P)*(sig*eps*b**2)-b*teta/P
    V = numpy.roots([1,alfa,beta,gamma])
    Vl = min(V)
    Vv = max(V)
    return (P*(Vl-Vv) + (T*dteta-teta)*(psia(Vl)-psia(Vv))+T*(R*numpy.log((Vv-b)/(Vl-b))+dteta*(psia(Vv)-psia(Vl))))/(R*T)

psat = optimize.bisect(fps, Pinfi, Psupi, xtol=1e-12, rtol=4.4408920985006262e-16, maxiter=100, full_output=True, disp=True)

print("psat:",str(psat[0]) + ' bar')



plateaus: [ 4.25591143 -0.10653629  0.14642616  0.09212116]
Roots: [ 4.25591143  0.14642616]
Vli: 0.14642616259
Vvi: 4.25591142877
Psupi: 2.09228234169
Pinfi: 1e-09
psat: 0.001735215064619947 bar


# External references
scipy, optimize [documentation](http://docs.scipy.org/doc/scipy/reference/optimize.html)

# Credits
* Initially developed in python 2.7 by Guilherme Carneiro Queiroz da Silva
* Adapted to ipynb with python 3.5 by Iuri Soter Viana Segtovich